Author:
    - Name: Srikanth M
    
    
    

INTRODUCTION:


The code is expected to predict whether a given consumable is compatible or not compatible. In the case of compatible prediction, 
the code is expected to further classify the type of consumable used. If the second prediction is of required confidence, it is further
used to retrain the model and save the model in onnx format.


There are three stages:
1. stage I : classifies whether an consumable is compatible or not.
2. stage II : classifies the type of consumable
3. stage III: retrain the model 

To install the necessary packages,use the following pip commands:

pip install matplotlib pandas numpy tensorflow opencv-python pillow

pip install --upgrade tensorflow

pip install torch torchvision

pip install tensorflow keras onnx tf2onnx

pip install onnxruntime


Import the following libraries:

In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2
import PIL.Image as Image
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

ModuleNotFoundError: No module named 'matplotlib'

In [3]:
import imghdr

C:\Users\srik7\AppData\Local\Temp\ipykernel_4708\336752527.py:1: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [5]:
import onnx
import tf2onnx

In [6]:
import onnxruntime as ort

In [7]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18


STAGE I CLASSIFCATION MODEL:

Iterate through the data directory containing multiple classes of images, checking each image file for its validity based on its extension. 
The data directory contains the data for stage I classification. 
It leads to a folder containing two folders:
1. valid consumable
2. invalid consumable


In [ ]:
image_exts = ['jpeg', 'jpg', 'bmp', 'png']
data_dir_1 = 'newdata\stage1' # data for stage one classificaton
for image_class in os.listdir(data_dir_1):
    for image in os.listdir(os.path.join(data_dir_1, image_class)):
        image_path = os.path.join(data_dir_1, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print("Image not in extension list{}".format(image_path))
                os.remove(image_path)
        except Exception as e:
            print("Issue with image {}".format(image_path))

Prepare the stage I data:

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir_1)
data_iterator = iter(data)
batch = data_iterator.next()
scaled =batch[0]/255
data = data.map(lambda x,y : (x/255, y))
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)+1
test_size = int(len(data)*.1)+1
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test =data.skip(train_size+val_size).take(test_size)

Stage I model:

 A simple convolutional neural network (CNN) architecture using Keras (tf.keras) for classifying consumable items based on the provided layers:

Usage:

This model structure is suitable for binary classification tasks where the input images are 256x256 pixels with 3 color channels (RGB).

Compile and train the model using appropriate optimizer, loss function, and training data to fit your specific classification problem.

In [ ]:
validconsumablemodel = Sequential()

In [ ]:
validconsumablemodel.add(Conv2D(16,(3,3), 1, activation = 'relu', input_shape = (256,256,3)))
validconsumablemodel.add(MaxPooling2D())

validconsumablemodel.add(Conv2D(32,(3,3), 1, activation = 'relu'))
validconsumablemodel.add(MaxPooling2D())

validconsumablemodel.add(Conv2D(16,(3,3), 1, activation = 'relu'))
validconsumablemodel.add(MaxPooling2D())

validconsumablemodel.add(Flatten())

validconsumablemodel.add(Dense(256, activation = 'relu'))
validconsumablemodel.add(Dense(1, activation = 'sigmoid'))

compile the model:

This script sets up a simple CNN architecture for binary classification of images (256x256 pixels with 3 color channels).
Compiles the model with Adam optimizer and binary cross-entropy loss.
Provides a model summary for review and verification of the architecture and parameter counts before training. Adjust the model architecture and hyperparameters as needed for your specific classification task.

Usage:

Ensure to compile the model after defining its architecture and before training it on your dataset.

Use model.fit() to train the compiled model with your training data.

In [ ]:
validconsumablemodel.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics = ['accuracy'])
validconsumablemodel.summary()

Train the model:

This setup enables you to track and visualize your model's training progress and performance using TensorBoard.
Adjust the logdir path to a directory where you want to store TensorBoard logs.
Use %time to monitor training time directly within Jupyter/IPython notebooks.
Usage:

Ensure you have TensorFlow installed (pip install tensorflow).
Set up the train and val datasets appropriately with features and labels for training and validation.
Adjust epochs based on your training needs.
View TensorBoard logs by running tensorboard --logdir=log in your terminal and navigating to http://localhost:6006 in your web browser.


In [ ]:
logdir='log' #log directory
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)

hist = validconsumablemodel.fit(train, epochs = 23, validation_data=val, callbacks = [tensorboard_callback])

In [ ]:
def plot_loss(history):
    """
    Plot the training and validation loss from the history object.
    
    Parameters:
    - history (dict): A dictionary containing training and validation loss values.
                      Typically obtained from a training process where metrics
                      like loss are recorded after each epoch.
    
    Returns:
    - None: Displays a plot using Matplotlib.
    
    Example usage:
    >>> plot_loss(history)
    """
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
      
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.legend()
    plt.grid(True)

In [ ]:
plot_loss(hist)

SAVE THE MODEL:

The model is saved into two formats:
1. h5
2. onnx

In [ ]:


validconsumablemodel.save(os.path.join('models','stage1validconsumableclassifier.h5'))

In [ ]:
validconsumablemodel.output_names=['output']
# Convert the Keras model to ONNX format
spec = (tf.TensorSpec((None, 256, 256, 3), tf.float32, name="input"),)
onnx_model, _ = tf2onnx.convert.from_keras(validconsumablemodel, input_signature=spec, opset=13)

# Create the directory if it doesn't exist
os.makedirs('models', exist_ok=True)
validconsumablemodel.output_names=['output']
# Save the ONNX model to a file
onnx_save_path = 'models/stage1validconsumableclassifier.onnx'
with open(onnx_save_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"Model has been converted to ONNX format and saved as '{onnx_save_path}'")

LOAD THE MODEL:

keras model:

In [14]:
from tensorflow.keras.models import load_model
model_path = 'models\stage1validconsumableclassifier.h5' #path

# Load the model
validconsumablemodel_loaded = load_model(model_path)

# Optional: Print model summary
validconsumablemodel_loaded.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 14400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     3,686,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,696,627 (14.10 MB)

 Trainable params: 3,696,625 (14.10 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

onnx model:

In [ ]:
import onnxruntime

# Load the ONNX model
onnx_model_path = 'models/stage1validconsumableclassifier.onnx'  # Replace with your ONNX model path
session = onnxruntime.InferenceSession(onnx_model_path)

# Check the input names of the ONNX model (optional)
input_name = session.get_inputs()[0].name
print(f'Input name: {input_name}')

# Check the output names of the ONNX model (optional)
output_name = session.get_outputs()[0].name
print(f'Output name: {output_name}')

STAGE II CLASSIFICATION MODEL:

(skip this part if you have enough data)
Define a function,"duplicate_image", which take in an image, output folder path and number of samples.
The copies vary in axis, a 7 degree magnitude of rotation and brightness.
The samples are stored in specified folder.

In [ ]:
def duplicate_image(image_path, output_folder, num_copies=100):
    """
    Duplicate an image multiple times with random transformations and save the augmented images to a specified folder.

    Parameters:
    - image_path (str): Path to the input image file.
    - output_folder (str): Path to the folder where augmented images will be saved.
    - num_copies (int, optional): Number of augmented copies to create (default is 100).

    Returns:
    - None

    Notes:
    - If the specified output_folder does not exist, it will be created.
    - Each duplicated image will be saved with a unique filename in the format "augmented_image_i.jpg".

    Example:
    >>> duplicate_image('input_image.jpg', 'augmented_images', num_copies=50)
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image = Image.open(image_path)

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=7),
        transforms.ColorJitter(brightness=(0.5, 1.5))
    ])

    for i in range(num_copies):
        transformed_image = transform(image)
        transformed_image.save(os.path.join(output_folder, f"augmented_image_{i + 1}.jpg"))



Data directoy and output class names are defined.

Data expected:
* compatible consumables divided into their corresponding folders

In [ ]:
data_dir = 'newdata\stage2' #path
CLASS_NAMES= ['130A','50A','90A']

TRANSFORMS:

Two transforms are created. One for training dataset and one for testing dataset.
The transform used for training dataset, resizes, flips, rotates and converts the image to a tensor.
the tranform used for testing dataset resizes and converts it to tensors.

In [ ]:
training_transforms = transforms.Compose([transforms.Resize((256,256)),
                                          transforms.RandomVerticalFlip(),
                                          transforms.RandomRotation(7),
                                          transforms.ToTensor()
                                          ])
testing_transforms = transforms.Compose([transforms.Resize((256,256)),                                                                                  
                                         transforms.ToTensor()
                                          ])

DATASET CREATION:

Two datasets, are created and transformed with their corresponding transforms.

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(root = data_dir, transform = training_transforms )
test_dataset = torchvision.datasets.ImageFolder(root = data_dir, transform = testing_transforms )

These data sets are then fed into a loader, which loads them in specified batches. In this case, it is batches of 23 images.
* The train loader shuffles the dataset
* the test loader does not shuffle the dataset

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = 23, shuffle = True )
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = 23, shuffle = False )

A function, "show_transformed_images(dataset)",  to visualize the loader data:

In [ ]:
def show_transformed_images(dataset):
    """
    Display a grid of transformed images from a given dataset.

    Parameters:
    - dataset (torch.utils.data.Dataset): Torch dataset object containing images and labels.

    Returns:
    - None

    Example:
    >>> show_transformed_images(my_dataset)
    """
    loader = torch.utils.data.DataLoader(dataset, batch_size = 6, shuffle = True)
    batch = next(iter(loader))
    images, labels = batch


    grid = torchvision.utils.make_grid(images, nrow = 3)
    plt.figure(figsize = (11,12))
    plt.imshow(np.transpose(grid, (1,2,0)))
    print('labels: ', labels)

In [ ]:
show_transformed_images(train_dataset)

The function set_device() checks if a CUDA-enabled GPU is available on your system. If CUDA is available, it returns a torch.device object representing the GPU ('cuda:0'), specifically the first GPU if there are multiple. If CUDA is not available (i.e., no GPU is available or CUDA drivers are not properly configured), it returns a torch.device object representing the CPU ('cpu').

In [16]:
def set_device():
    """
    Set the device for PyTorch computations based on GPU availability.

    Returns:
    - torch.device: CUDA device ('cuda:0') if GPU is available, otherwise CPU device ('cpu').

    Example:
    >>> device = set_device()
    >>> print(device)
    cuda:0  # Example output when GPU is available
    """
    if torch.cuda.is_available():
        dev = 'cuda:0'
    else:
        dev = 'cpu'

    return torch.device(dev)

TRAINING THE MODEL:


This train_nn function is designed to train a neural network model using PyTorch, iterating over a specified number of epochs (n_epochs). 

In [ ]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs):
    """
    Train a neural network model using PyTorch.

    Parameters:
    - model (torch.nn.Module): The neural network model to be trained.
    - train_loader (torch.utils.data.DataLoader): DataLoader for the training dataset.
    - test_loader (torch.utils.data.DataLoader): DataLoader for the test dataset.
    - criterion: Loss function to optimize the model.
    - optimizer: Optimization algorithm to update the model parameters.
    - n_epochs (int): Number of epochs (iterations over the entire dataset) for training.

    Returns:
    - torch.nn.Module: Trained neural network model.

    Notes:
    - The function prints training progress and evaluation results after each epoch.
    - It evaluates the model's performance on the test set using a separate function `evaluate_model_on_test_set`.

    Example:
    >>> trained_model = train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs=10)
    """
    device = set_device()
    for epoch in range(n_epochs):
        print('Epoch number %d' %(epoch +1))
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        for data in train_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            optimizer.zero_grad()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()
        epoch_loss = running_loss/len(train_loader)
        epoch_acc = 100.00*running_correct/total
        print("-----Training dataset, identified %d out of %d images correctly (%.3f%%). epoch loss: %.3f" % (running_correct, total, epoch_acc, epoch_loss))
        evaluate_model_on_test_set(model, test_loader) #Calls a function (evaluate_model_on_test_set) to evaluate the model's performance on the test set after each epoch.
    print('finished')
    return model
    


The evaluate_model_on_test_set function is designed to evaluate a trained neural network model using a test dataset.

In [ ]:
def evaluate_model_on_test_set(model, test_loader):
    model.eval()
    predicted_correctly_on_epoch = 0
    total = 0
    device = set_device()
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predicted_correctly_on_epoch += (predicted == labels).sum().item()
    
            epoch_acc = 100.00 * predicted_correctly_on_epoch/total
           
            print(' --------testing dataset, identified %d out of %d images correctly (%.3f%%)' %(predicted_correctly_on_epoch, total, epoch_acc))

set up a neural network for image classification using the ResNet-18 architecture pretrained on ImageNet. 

This code snippet initializes a ResNet18 model pre-trained on ImageNet (consumable_classifier) and modifies its fully connected (FC) layer for a custom classification task with 3 classes. It then moves the model to the appropriate device (GPU or CPU), sets up a cross-entropy loss function, and defines an Adam optimizer for training.

In [ ]:
consumable_classifier  = models.resnet18(pretrained = True)
num_features = consumable_classifier.fc.in_features
number_of_classes = 3
consumable_classifier.fc = torch.nn.Linear(num_features, number_of_classes)

device = set_device()
consumable_classifier = consumable_classifier.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(consumable_classifier.parameters(), lr=0.001)

Train the model for 5 epochs

In [ ]:
train_nn(consumable_classifier, train_loader, test_loader, loss_function, optimizer, 5)

Set up an image transform to test unseen data:

In [30]:
image_transform = transforms.Compose([transforms.Resize((256,256)),
                                         transforms.ToTensor()
                                          ])


Function,"classify(model,image_transforms,image_path,classes)", takes a single image and classifys it accordingly.


In [18]:
def classify(model,image_transforms,image_path,classes):
    """
    Classify an image using a PyTorch model.

    Parameters:
    - model (torch.nn.Module): The trained neural network model to perform inference.
    - image_transforms (torchvision.transforms.Compose): Image transformations to be applied.
    - image_path (str): Path to the input image file.
    - classes (list): List of class labels corresponding to the model's output.

    Returns:
    - None

    Prints:
    - str: Predicted class label based on the highest model output probability.

    Example:
    >>> from torchvision import transforms
    >>> model = ...  # Load your trained model
    >>> image_transforms = transforms.Compose([
    >>>     transforms.Resize((256, 256)),
    >>>     transforms.ToTensor(),
    >>> ])
    >>> image_path = 'path_to_your_image.jpg'
    >>> classes = ['class1', 'class2', 'class3']
    >>> classify(model, image_transforms, image_path, classes)
    """

    model = model.eval()
    image = Image.open(image_path)
    image = image_transforms(image).float()
    image = torch.unsqueeze(image,0)
    output = model(image)
    _,prediction =torch.max(output.data,1)
    print(classes[prediction.item()])  

SAVE THE MODEL:

In [ ]:
torch.save(consumable_classifier.state_dict(), os.path.join('models',"consumable_classifier_stage2.pth.tar")) #add the path name and model name

lOAD THE MODEL:

In [20]:
file_path = 'models\consumable_classifier_stage2.pth.tar' #specify correct path

consumable_classifier_model_load = models.resnet18(num_classes=3)  

state_dict = torch.load(file_path)


consumable_classifier_model_load.load_state_dict(state_dict)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
consumable_classifier_model_load = consumable_classifier_model_load.to(device)

STAGE III CLASSIFY, RETRAIN AND SAVE:

In [66]:
def classication(model_a,model_b,image_path):
    """
    Classifies an image using two sequential models and performs additional actions based on the predictions.

    Parameters:
    - model_a: TensorFlow or Keras model for initial classification.
    - model_b: PyTorch model for secondary classification and potential retraining.
    - image_path (str): Path to the image file to classify.

    Returns:
    - None: Prints results of initial classification and potential retraining.

    Example usage:
    >>> model_a = tf.keras.models.load_model('model_a.h5')
    >>> model_b = load_torch_model('model_b.pth')
    >>> image_path = './image.jpg'
    >>> classification(model_a, model_b, image_path)
    """
    img = cv2.imread(image_path)
    resize = tf.image.resize(img,(256,256))
    yhat = model_a.predict(np.expand_dims(resize/255,0))
    print(yhat)
    if yhat>0.7:
        print('consumable compatible')
        image_transform = transforms.Compose([transforms.Resize((256,256)),
                                         transforms.ToTensor()
                                          ])
        CLASS_NAMES= ['130A','50A','90A']
        yhat_2 = classify_stage2(model_b,image_transform,image_path,CLASS_NAMES)
        if yhat_2> 0.85:
            retrain_single_image(model_b, image_path, CLASS_NAMES, num_epochs=1,save_path="updatedmodel.pth.tar") #specify save path
    else:
        print('consumable not compatible')

In [24]:
def classify_stage2(model,image_transforms,image_path,classes):
    """
    Performs a secondary classification on an image using a PyTorch model and prints the predicted class and probability.

    Parameters:
    - model: PyTorch model used for secondary classification.
    - image_transforms (torchvision.transforms.Compose): Transformations to apply to the image before inference.
    - image_path (str): Path to the image file to classify.
    - classes (list): List of class labels corresponding to the model's output classes.

    Returns:
    - float: Probability of the predicted class.

    Example usage:
    >>> model = load_torch_model('model.pth')
    >>> image_transforms = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
    >>> image_path = './image.jpg'
    >>> classes = ['cat', 'dog', 'rabbit']
    >>> probability = classify_stage2(model, image_transforms, image_path, classes)
    """
    model = model.eval()
    image = Image.open(image_path)
    image = image_transforms(image).float()
    image = torch.unsqueeze(image,0)
    output = model(image)
    _,prediction =torch.max(output.data,1)
    probabilities = F.softmax(output, dim=1)
    print(f'Predicted class: {classes[prediction]}')
    print(f'Predicted class probability: {probabilities[0][prediction.item()].item()}')
    
    return probabilities[0][prediction.item()].item()

In [42]:
def retrain_single_image(model, image_path, classes, num_epochs, save_path):
    """
    Retrains a PyTorch model using a single image, saves the updated model,
    and exports it to ONNX format for deployment.

    Parameters:
    - model: PyTorch model to be retrained.
    - image_path (str): Path to the image used for retraining.
    - classes (list): List of class labels corresponding to the model's output classes.
    - num_epochs (int): Number of epochs to train the model.
    - save_path (str): Path to save the retrained model's state_dict and ONNX file.

    Returns:
    - None: Prints training progress and saves the retrained model.

    Example usage:
    >>> model = load_pretrained_model('pretrained_model.pth')
    >>> image_path = './image.jpg'
    >>> classes = ['cat', 'dog', 'rabbit']
    >>> num_epochs = 10
    >>> save_path = 'retrained_model.pth.tar'
    >>> retrain_single_image(model, image_path, classes, num_epochs, save_path)
    """
    def preprocess_image(image_path):
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),  # Ensure the image is of the correct size
            transforms.ToTensor()
        ])
        image = Image.open(image_path)
        return preprocess(image).unsqueeze(0)

    
    def predict_image(model, image_path):
        image = preprocess_image(image_path)
        model.eval()
        with torch.no_grad():
            output = model(image)
            _, predicted_class = torch.max(output, 1)
        return predicted_class.item()

    
    image_tensor = preprocess_image(image_path)

    
    predicted_class = predict_image(model, image_path)
    print(f'Predicted class: {classes[predicted_class]}')

    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    image_tensor = image_tensor.to(device)
    label_tensor = torch.tensor([predicted_class], dtype=torch.long).to(device)

    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(image_tensor)
        loss = criterion(outputs, label_tensor)
        loss.backward()
        optimizer.step()
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

    print("Model retrained and updated")

    # Save the PyTorch model
    os.makedirs('models', exist_ok=True)
    torch.save(model.state_dict(), os.path.join('models/updatedmodels', save_path))
    print(f'Model saved to {os.path.join("models/updatedmodels", save_path)}')

    # Save the model in ONNX format
    model.eval()
    onnx_save_path = os.path.splitext(save_path)[0] + '.onnx'
    dummy_input = torch.randn(1, 3, 256, 256).to(device)
    torch.onnx.export(
        model,
        dummy_input,
        os.path.join('models/updatedmodels', onnx_save_path),
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
    )
    print(f'Model saved to ONNX format')

LOAD THE UPDATED MODEL:

PyTorch Model:

In [109]:
classication(validconsumablemodel_loaded,consumable_classifier_model_load,'50A_test.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
[[0.97969407]]
consumable compatible
Predicted class: 50A
Predicted class probability: 0.9999967813491821
Predicted class: 50A
Epoch [1/1], Loss: 1.2938257455825806
Model retrained and updated
Model saved to models/updatedmodels\updatedmodel.pth.tar
Model saved to ONNX format


In [98]:
file_path = 'models/updatedmodels/updatedmodel.pth.tar'
#file path for pytorch model

model_d = models.resnet18(num_classes=3)  

state_dict_1 = torch.load(file_path)


model_d.load_state_dict(state_dict)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_d = model_d.to(device)

onnx model:

In [ ]:
# Load the ONNX model
onnx_model_path = "models/updatedmodels/updatedmodels.pth.onnx" #file path for onnx model
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

#  Create an inference session
session = ort.InferenceSession(onnx_model_path)


In [ ]:
def preprocess_image(image_path):
    """
    Preprocesses an image for a neural network model.

    Parameters:
    - image_path (str): Path to the input image file.

    Returns:
    - numpy.ndarray: Preprocessed image as a numpy array with shape (1, 3, 224, 224).

    Example:
    >>> image_path = 'path_to_your_image.jpg'
    >>> preprocessed_image = preprocess_image(image_path)
    """
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        
    ])
    
    image = Image.open(image_path)
    image = preprocess(image)
    return image.unsqueeze(0).numpy()


example use of the loaded onnx model:

In [ ]:
image_path ="90A_1.jpg" #image to be classified
input_data = preprocess_image(image_path)
CLASS_NAMES= ['130A','50A','90A']#output class names
# Get the name of the input node
input_name = session.get_inputs()[0].name


outputs = session.run(None, {input_name: input_data})

# Get the output
output_name = session.get_outputs()[0].name
predicted_output = outputs[0]

# Post-process the output (e.g., get the predicted class)
predicted_class = np.argmax(predicted_output, axis=1)
print(f'Predicted class: {CLASS_NAMES[predicted_class[0]]}')

example use of the loaded model:

In [113]:

classify_stage2(consumable_classifier_model_load,image_transform,"50A_test.jpg",CLASS_NAMES)


Predicted class: 50A
Predicted class probability: 0.9999958276748657


0.9999958276748657

In [115]:
classify_stage2(model_d,image_transform,"50A_test.jpg",CLASS_NAMES)

Predicted class: 50A
Predicted class probability: 0.9999123811721802


0.9999123811721802